In [221]:
import pandas as pd
import numpy as np
from datetime import datetime
import math 

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [411]:
# Import xlsx file and store each sheet in to a df list
xl_file = pd.ExcelFile('./data.xlsx',)

dfs = {sheet_name: xl_file.parse(sheet_name) 
          for sheet_name in xl_file.sheet_names}

In [420]:
# Data from each sheet can be accessed via key
keyList = list(dfs.keys())

In [412]:
# Data cleansing
for key, df in dfs.items():
    dfs[key].loc[:,'Confirmed'].fillna(value=0, inplace=True)
    dfs[key].loc[:,'Deaths'].fillna(value=0, inplace=True)
    dfs[key].loc[:,'Recovered'].fillna(value=0, inplace=True)
    dfs[key]=dfs[key].astype({'Confirmed':'int64', 'Deaths':'int64', 'Recovered':'int64'})
    # Change as China for coordinate search
    dfs[key]=dfs[key].replace({'Country/Region':'Mainland China'}, 'China')
    dfs[key]=dfs[key].replace({'Province/State':'Queensland'}, 'Brisbane')
    dfs[key]=dfs[key].replace({'Province/State':'New South Wales'}, 'Sydney')
    dfs[key]=dfs[key].replace({'Province/State':'Victoria'}, 'Melbourne')
    # Add a zero to the date so can be convert by datetime.strptime as 0-padded date
    dfs[key]['Last Update'] = '0' + dfs[key]['Last Update']
    dfs[key]['Date_last_updated'] = [datetime.strptime(d, '%m/%d/%Y %H:%M') for d in dfs[key]['Last Update']]

In [413]:
# Check 
dfs['2020-01-29-21-00'].tail()

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Date_last_updated
51,Melbourne,Australia,01/29/2020 21:00,2,0,0,2020-01-29 21:00:00
52,Brisbane,Australia,01/29/2020 21:00,1,0,0,2020-01-29 21:00:00
53,Bavaria,Germany,01/29/2020 21:00,4,0,0,2020-01-29 21:00:00
54,NaN,Finland,01/29/2020 21:00,1,0,0,2020-01-29 21:00:00
55,NaN,United Arab Emirates,01/29/2020 21:00,4,0,0,2020-01-29 21:00:00


In [414]:
# Add coordinates for each area in the list for the latest table sheet
# As there are limit for free account, we only call coordinates for the latest table sheet
from opencage.geocoder import OpenCageGeocode
key = 'b33700b33d0a446aa6e16c0b57fc82d1'  # get api key from:  https://opencagedata.com

list_lat = []   # create empty lists
list_long = []    
for index, row in dfs[keyList[0]].iterrows(): # iterate over rows in dataframe
    City = row['Province/State']
    State = row['Country/Region']

    if City:
        if City == 'Macau' or City == 'Hong Kong':
            query = str(City)+','+'China'
            results = geocoder.geocode(query)   
            lat = results[0]['geometry']['lat']
            long = results[0]['geometry']['lng']

            list_lat.append(lat)
            list_long.append(long)
        else:
            query = str(City)+','+str(State)
            results = geocoder.geocode(query)   
            lat = results[0]['geometry']['lat']
            long = results[0]['geometry']['lng']

            list_lat.append(lat)
            list_long.append(long)

    else:
        query = str(State)
        results = geocoder.geocode(query)   
        lat = results[0]['geometry']['lat']
        long = results[0]['geometry']['lng']

        list_lat.append(lat)
        list_long.append(long)
# create new columns from lists    
dfs[keyList[0]]['lat'] = list_lat   
dfs[keyList[0]]['lon'] = list_long

In [415]:
dfs[keyList[0]]

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Date_last_updated,lat,lon
0,Hubei,China,01/29/2020 21:00,4586,162,90,2020-01-29 21:00:00,31.151725,112.878322
1,Zhejiang,China,01/29/2020 21:00,428,0,4,2020-01-29 21:00:00,29.000000,120.000000
2,Guangdong,China,01/29/2020 21:00,311,0,6,2020-01-29 21:00:00,23.135769,113.198269
3,Henan,China,01/29/2020 21:00,278,2,2,2020-01-29 21:00:00,34.000000,114.000000
4,Hunan,China,01/29/2020 21:00,277,0,0,2020-01-29 21:00:00,27.666209,111.748706
5,Anhui,China,01/29/2020 21:00,200,0,2,2020-01-29 21:00:00,32.000000,117.000000
6,Chongqing,China,01/29/2020 21:00,165,0,1,2020-01-29 21:00:00,29.558571,106.549282
7,Jiangxi,China,01/29/2020 21:00,162,0,4,2020-01-29 21:00:00,28.000000,116.000000
8,Shandong,China,01/29/2020 21:00,145,0,1,2020-01-29 21:00:00,36.000000,119.000000
9,Sichuan,China,01/29/2020 21:00,142,1,1,2020-01-29 21:00:00,30.500000,102.500000


In [416]:
# Construct new dataframe for line plot
DateList = []
ChinaList =[]
OtherList = []

for key, df in dfs.items():
    dfTpm = df.groupby(['Country/Region'])['Confirmed'].agg(np.sum)
    dfTpm = pd.DataFrame({'Code':dfTpm.index, 'Confirmed':dfTpm.values})
    dfTpm = dfTpm.sort_values(by='Confirmed', ascending=False).reset_index(drop=True)
    DateList.append(df['Date_last_updated'][0])
    ChinaList.append(dfTpm['Confirmed'][0])
    OtherList.append(dfTpm['Confirmed'][1:].sum())
    
df_confirmed = pd.DataFrame({'Date':DateList,
                             'Mainland China':ChinaList,
                             'Other locations':OtherList})    


In [417]:
df_confirmed.head()

,Date,Mainland China,Other locations
0,2020-01-29 21:00:00,7678,105
1,2020-01-29 14:30:00,6070,95
2,2020-01-29 13:30:00,6070,94
3,2020-01-28 23:00:00,5970,87
4,2020-01-28 18:00:00,5494,84


In [518]:
# Set up tick scale based on confirmed case number
tickList = list(np.arange(0, df_confirmed['Mainland China'].max()+1000, 1000))

# Create empty figure canvas
fig = go.Figure()
# Add trace to the figure
fig.add_trace(go.Scatter(x=df_confirmed['Date'], y=df_confirmed['Mainland China'],
                         mode='lines+markers',
                         name='Mainland China',
                         line=dict(color='#fe582c', width=2),
                         marker=dict(size=8),
                         text=[datetime.strftime(d, '%b %d %Y %H:%M EST') for d in df_confirmed['Date']],
                         hovertemplate='<b>%{text}</b><br></br>'+
                                       'Mainland China confirmed<br>'+
                                       '%{y} cases<br>'+
                                       '<extra></extra>'))
fig.add_trace(go.Scatter(x=df_confirmed['Date'], y=df_confirmed['Other locations'],
                         mode='lines+markers',
                         name='Other Region',
                         line=dict(color='#2cfec1', width=2),
                         marker=dict(size=8),
                         text=[datetime.strftime(d, '%b %d %Y %H:%M EST') for d in df_confirmed['Date']],
                         hovertemplate='<b>%{text}</b><br></br>'+
                                       'Other region confirmed<br>'+
                                       '%{y} cases<br>'+
                                       '<extra></extra>'))
# Customise layout
fig.update_layout(
    title=dict(
        text="Total Confirmed Cases",
        y=0.95, x=0.5, xanchor='center', yanchor='top',
        font=dict(size=20, color="#7fafdf")
    ),
    margin=go.layout.Margin(
        l=50,
        r=10,
        b=10,
        t=60,
        pad=0
    ),
    yaxis=dict(
        showline=True, linecolor='#272e3e',
        zeroline=False,
        gridcolor='#272e3e',
        gridwidth = .1,
        tickmode='array',
        # Set tick range based on the maximum number
        tickvals=tickList,
        # Set tick label accordingly
        ticktext=['{:.0f}k'.format(i/1000) for i in tickList]
    ),
    yaxis_title="Confirmed case number",
    xaxis=dict(
        showline=True, linecolor='#272e3e',
        gridcolor='#272e3e',
        gridwidth = .1,
        zeroline=False
    ),
    xaxis_tickformat='%b %d',
    legend_orientation="h",
    plot_bgcolor='#151920',
    paper_bgcolor='#272e3e',
    font=dict(color='#7fafdf')
)

fig.show()

In [519]:
mapbox_access_token = "pk.eyJ1IjoicGxvdGx5bWFwYm94IiwiYSI6ImNqdnBvNDMyaTAxYzkzeW5ubWdpZ2VjbmMifQ.TXcBE-xg9BFdV2ocecc_7g"

textList=[]
for area, region in zip(dfs[keyList[0]]['Province/State'], dfs[keyList[0]]['Country/Region']):
    
    if type(area) is str:
        if region == "Hong Kong" or region == "Macau" or region == "Taiwan":
            textList.append(area)
        else:
            textList.append(area+', '+region)
    else:
        textList.append(region)

fig2 = go.Figure(go.Scattermapbox(
        lat=dfs[keyList[0]]['lat'],
        lon=dfs[keyList[0]]['lon'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            color='#ff1414',
            size=dfs[keyList[0]]['Confirmed'].tolist(), 
            sizemin=2,
            sizemode='area',
            sizeref=2.*max(dfs[keyList[0]]['Confirmed'].tolist())/(40.**2),
        ),
        text=textList,
        hovertext=['Comfirmed: {}<br>Death: {}<br>Recovered: {}'.format(i, j, k) for i, j, k in zip(dfs[keyList[0]]['Confirmed'],
                                                                                                    dfs[keyList[0]]['Deaths'],
                                                                                                    dfs[keyList[0]]['Recovered'])],
    
        hovertemplate = "<b>%{text}</b><br><br>" +
                        "%{hovertext}<br>" +
                        "<extra></extra>")
    
        )

fig2.update_layout(
    plot_bgcolor='#151920',
    paper_bgcolor='#272e3e',
    margin=go.layout.Margin(
        l=10,
        r=10,
        b=10,
        t=10,
        pad=40
    ),
    hovermode='closest',
    mapbox=go.layout.Mapbox(
        accesstoken=mapbox_access_token,
        style="mapbox://styles/plotlymapbox/cjvprkf3t1kns1cqjxuxmwixz",
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=23.1301964,
            lon=113.2592945
        ),
        pitch=0,
        zoom=3
    )
)

fig2.show()

In [340]:
import dash
import dash_core_components as dcc
import dash_html_components as html 
import dash_bootstrap_components as dbc

In [459]:
app = dash.Dash(__name__, assets_folder='./assets/')

In [601]:
app.layout = html.Div(style={'backgroundColor':'#151920'},
    children=[
        html.Div(
            id="header",
            children=[
                html.H4(children="Wuhan Coronavirus (2019-nCoV) Outbreak Monitor"),
                html.P(
                    id="description",
                    children="On December 31, 2019, the World Health Organization (WHO) was informed of \
                    an outbreak of “pneumonia of unknown cause” detected in Wuhan City, Hubei Province, China – the \
                    seventh-largest city in China with 11 million residents. As of January 23, there are over 800 cases \
                    of 2019-nCoV confirmed globally, including cases in at least 20 regions in China and nine countries/territories.\
                    This dash board is developed to visualise and track the recent reported cases on a daily timescale.",
                ),
            ]        
        ),
        html.Div(
            children=[
                html.Div(style={'width':'50%','display':'inline-block','marginLeft':'1.5%'},
                    children=[
                        html.Div(dcc.Graph(figure=fig),
                                 style={'display':'inline-block','marginRight':'1.5%'}),
                        html.Div(dcc.Graph(figure=fig),
                                 style={'display':'inline-block','marginRight':'1.5%'})
                    ]
                ),           
                html.Div(style={'display':'inline-block','marginLeft':'1.5%'},
                    children=[dcc.Graph(figure=fig2)])
            ]
        )
    ])  
    

In [ ]:
if __name__ == '__main__':
    app.run_server(port=8882)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8882/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Jan/2020 21:59:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Jan/2020 21:59:07] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.1.2&m=1576595738 HTTP/1.1" 200 -
127.0.0.1 - - [30/Jan/2020 21:59:07] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.1.2&m=1576595738 HTTP/1.1" 200 -
127.0.0.1 - - [30/Jan/2020 21:59:07] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.1.2&m=1576595738 HTTP/1.1" 200 -
127.0.0.1 - - [30/Jan/2020 21:59:07] "GET /_dash-component-suites/dash_html_components/dash_html_components.min.js?v=1.0.1&m=1576596177 HTTP/1.1" 200 -
127.0.0.1 - - [30/Jan/2020 21:59:07] "GET /_dash-component-suites/dash_core_components/highlight.pack.js?v=1.3.1&m=1576595950 HTTP/1.1" 200 -
127.0.0.1 - - [30/Jan/2020 21:59:07] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.min.js?v=0.8.2&m=1580284437 HTTP/1.1